El objetivo es encotrar la informacion mas posible, y que sea de calidad. De acuerdo a los criterios de PLD

| Información requerida                   | Información obtenida                              |
|-----------------------------------------|--------------------------------------------------|
| negocio                                 |                                                  |
| nombre cliente o razón social           |    BANOM_BAN                                    |
| número cliente                          |    BACVE_BANC                                     |
| estatus                                 |     WHERE BA_ACTIVO = 'S'              |
| número contrato                         |                                                  |
| género                                  |                N/A                                  |
| fecha_nacimiento o de constitución      |                                                  |
| entidad federativa nacimiento           |                                                  |
| país de nacimiento                      |    BACVE_PAIS_SHCP   PAIS DE CONSTITUCION    |
| nacionalidad                            |  BANACIONAL <br> IDENTIFICA SI EL BANCO ES NACIONAL O EXTRANJERO (N=EXTRANJERO  S=NACIONAL)                  |
| profesión                               |           N/A                                       |
| calle                                   |    PLDOM_CALLE                             |
| número_exterior                         |     PLDOM_NUMERO                                             |
| número_interior                         |                                                  |
| colonia_urbanización                    |                                                  |
| delegación_municipio                    |                                                  |
| ciudad_población                        |                    PLDOM_CIUDAD                              |
| entidad_federativa                      |             PLDOM_ESTADO                       |
| código postal                           |               PLDOM_CP                                   |
| país                                    |                PAPAIS                |
| teléfono                                |                                    |
| correo_electrónico                      |                                                  |
| rfc                                     |     |
| curp                                    |                                                  |
| fiel                                    |                                                  |
| representante legal                     |    NO SE CUENTA INFORMACION        |
| tipo de persona                         |  PM                                    |
| clasificación por grado de riesgo       |                                                  |
| pep (persona políticamente expuesta)    |                                                  |
| razón                                   |                                                  |
| producto contratado                     |                   |
| cambios en perfil transaccional         |                                                  |


LOS CRITERIOS DE DIRECCION SE OBTIENE MEDIANTE SOI_PLAZA QUE RELACIONA LAS PLAZAS POR BANCO 

In [3]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
from rapidfuzz import process, fuzz
import numpy as np

In [97]:
df = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/Clientes_activos_SOI.dat")

In [98]:
def limpiar_calle(calle):
    # Manejar valores NaN o nulos directamente
    if pd.isna(calle):
        return pd.NA
    
    # Convertir a string por seguridad
    calle = str(calle)
    
    # Procesar solo si contiene "--"
    if "--" in calle:
        return " -- ".join(sorted(set(c.strip() for c in calle.split("--") if c.strip() != 'NA')))
    
    # Devolver la calle procesada
    return calle.strip() 

In [99]:
df["CALLE"] = df["CALLE"].apply(lambda x: limpiar_calle(x))

In [100]:
df["NUMERO_EXTERIOR"] = df["NUMERO_EXTERIOR"].apply(lambda x: " -- ".join(sorted(set(c.strip() for c in x.split("--") if c.strip() != 'NA'))) if "--" in str(x) else x)

In [101]:
df["CIUDAD_POBLACION"] = df["CIUDAD_POBLACION"].apply(lambda x: " -- ".join(sorted(set(c.strip() for c in x.split("--") if c.strip() != 'NA'))) if "--" in str(x) else x)

In [102]:
df["ENTIDAD_FEDERATIVA"] = df["ENTIDAD_FEDERATIVA"].apply(lambda x: " -- ".join(sorted(set(c.strip() for c in x.split("--") if c.strip() != 'NA'))) if "--" in str(x) else x)


In [92]:
df["CODIGO_POSTAL"] = df["CODIGO_POSTAL"].apply(lambda x: " -- ".join(sorted(set(c.strip() for c in x.split("--") if c.strip() != 'NA'))) if "--" in str(x) else x)


In [103]:
df["RFC"] = df["RFC"].apply(lambda x: " -- ".join(sorted(set(c.strip() for c in x.split("--") if c.strip() != 'NA'))) if "--" in str(x) else x)

In [104]:
df["RFC"] = df["RFC"].apply(lambda x: normalizar_RFC(x) if pd.notna(x) and verificar_rfc(normalizar_RFC(x)) else "XXXXXXXXXXXXX")

In [105]:
df['FECHA_DE_ACTUALIZACION'] = datetime.now().strftime('%d/%m/%y')


In [106]:
col_final = ['NEGOCIO', 'NOMBRE_O_RAZON_SOCIAL', 'NUMERO_CLIENTE', 'ESTATUS',
       'NUMERO_CONTRATO', 'GENERO', 'FECHA_NAC_O_CONST',
       'ENTIDAD_FEDERATIVA_NACIMIENTO', 'PAIS_DE_NACIMIENTO', 'NACIONALIDAD',
       'PROFESION', 'CALLE', 'NUMERO_EXTERIOR', 'NUMERO_INTERIOR',
       'COLONIA_URBANIZACION', 'DELEGACION_MUNICIPIO', 'CIUDAD_POBLACION',
       'ENTIDAD_FEDERATIVA', 'CODIGO_POSTAL', 'PAIS', 'TELEFONO',
       'CORREO_ELECTRONICO', 'RFC', 'CURP', 'FIEL', 'REPRESENTANTE_LEGAL',
       'TIPO_PERSONA', 'PRODUCTO_CONTRATADO',
       'SISTEMA_ORIGEN', 'FECHA_DE_ACTUALIZACION']
df = df[col_final]

In [107]:
clt_repetidos = compara_nombres(df, df, "NOMBRE_O_RAZON_SOCIAL", "NOMBRE_O_RAZON_SOCIAL","NUMERO_CLIENTE","NUMERO_CLIENTE", 95)

In [114]:
clt_repetidos = compara_nombres(df, df, "NOMBRE_O_RAZON_SOCIAL", "NOMBRE_O_RAZON_SOCIAL","NUMERO_CLIENTE","NUMERO_CLIENTE", 95)
if not clt_repetidos.empty:
    df_resul, drops_ids = agregar_asociaciones_clientes(clt_repetidos,df)
    df = df.drop(drops_ids)
    df = pd.concat([df, df_resul], ignore_index=True)
    df = df.sort_values("NOMBRE_O_RAZON_SOCIAL")


In [110]:
name_archivo = os.path.join("/home/ale1726/proyects/datalake/clientes/data/ETL","CLientesT_SOI.csv")
df.to_csv(name_archivo,index=False)